# Training and testing model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from train import TrainingAlgorithm
from model import model
from utils import load_train_val_loaders

train_loader = load_train_val_loaders("..\data")

model_ = model()
losses = TrainingAlgorithm(model_, train_loader, num_epochs=10, device="cpu") #Include device? Think that is ok

epochs = np.arange(1, len(losses)+1)

plt.xlabel('Epoch')
plt.ylabel('Losses')
plt.yscale("log")
plt.plot(epochs, losses)
plt.title("Plotting total loss in each epoch")
plt.show()